# オプティマイズ
この版はパラメトリック最適化にするつもり。
- 中心周波数
- バンド幅
- 分散補償
計 3パラメーター

必要なものは
- スクイージングレベルの測定
    - ポンプ光のON/OFF arduinoモジュール OK
    - スペアナのSCPIコントロール
    - もしかしたらロック角度の変調
- waveshaperコントロール OK
- パラメーター生成 全探索 OK

In [29]:
import numpy as np
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
from mso58trans import MSO58transData
from shutter import shutter_handler
from sklearn.decomposition import PCA
import copy

In [ ]:
# 各種設定周り ここにすべてまとめる

class Setting:
    def __init__(self):
        self.ws_ip                          = '169.254.6.8'
        self.ws_Freq_resolution             = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange                   = None
        self.optSpectrum : Spectrum         = None
        self.mso58_address                  = 'USB::0x0699::0x0530::C043144::INSTR'
        self.homodyne_port                  = 1
        self.fastFramePulseNum              = 5
        self.quadrature_method              = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])
        self.targetLOPower                  = None
        self.shutter_con : shutter_handler  = shutter_handler(port='COM3')
        self.pumpShutterPort : int          = 2


    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None
        assert self.fastFramePulseNum        is not None
        assert self.quadrature_method        is not None
        assert self.targetLOPower            is not None
        assert self.shutter_con              is not None
        assert self.pumpShutterPort          is not None


setting = Setting()

from WSMethods import getWSrange
startFreq, endFreq    = getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum.band.frequency = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.band.frequency)

setting.condition_approval()

### 最適化フェーズ

In [27]:
class State:
    def __init__(self):
        self.current_param = None

In [4]:

def quad(w, pulseNum, quadrature_func):
    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))

    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]

    quadratures = quadrature_func(volts)
    return quadratures


def evalation(quadratures):
    return quadratures.var()



def param_gen(state: State):
    for center_wavelength in np.linspace(1543,1547,40):
        for band_width in np.linspace(0.2,1,20):
            for dispersion in [0]:
                yield center_wavelength, band_width, dispersion



def loop():
# optimizing loop
    for center_wavelength, band_width, dispersion in param_gen():
## new_LO_spectrum
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                                    centerWavelength    =center_wavelength,
                                    bandWidth           =band_width, band=band,
                                    dispersion          =dispersion
                                )
        compensated_Spectrum    = compensation(
                                    setting.optSpectrum,
                                    targetSpectrum      =filterSpectrum,
                                    band                =band
                                )
        total_power = calc_totalPower(
                                    setting.optSpectrum,
                                    compensated_Spectrum
                                )
## (power adjustment)
        filterSpectrum = attenuation(filterSpectrum, total_power - setting.targetLOPower)

## pump off
        setting.shutter_con.close([setting.pumpShutterPort])
## osc_com
        osc_handler = MSO58transData(setting.mso58_address)
        
        t, w = osc_handler.transfer2byte(setting.homodyne_port)
## quad
## eval_shotnoise

## pump on
## osc_com
## quad
## eval_squeeze

## recording



